In [1]:
import random, math
import pandas as pd
import numpy as np

In [2]:
# mean values
iat_RT_mean = 10.0
iat_NRT_mean = 5.0
st_RT_mean =2.0
st_NRT_mean = 4.0

data=[]


In [3]:
# Set initial conditions
RTCL = 3.0
nonRTCL=5.0
nRT =0
nNonRT=0
s=2
SCL=4.0

MC=0.0
rem_time=0
max_iter=200.00
random.seed(7)

In [4]:
def return_time(mean):
    r=random.random()
    time= -(mean)*math.log(r) 
    return round(time,4)

In [5]:
row=[round(MC,4),round(RTCL,4),round(nonRTCL,4),nRT,nNonRT,round(SCL,4),s,round(rem_time,4)]
data.append(row)
while MC <= max_iter:

    if SCL<RTCL and SCL<nonRTCL :
        MC=SCL
        
        if nRT!=0:
            nRT-=1
            s=1
            
            st_RT=return_time(st_RT_mean)
            SCL=MC+st_RT
            
        elif nNonRT!=0:
            nNonRT-=1
            s=2
            
            if rem_time!=0:
                SCL=MC+rem_time
                rem_time=0
            else:
                st_NRT=return_time(st_NRT_mean)
                SCL=MC+st_NRT
        else:
            s=0
            # Since system is in idle state, there is no SCL. 
            # SCL is being set to a large value for computation purpose.
            # Ideally, SCL will contain no value as no event is being served. 
            #SCL is being set to Nan in final output table
            SCL=max_iter+1000 
        
    elif RTCL == SCL:
        MC=RTCL
        iat_RT=return_time(iat_RT_mean)        
        RTCL=MC+iat_RT
        s=1
        
        st_RT=return_time(st_RT_mean)
        SCL=MC+st_RT
        
        
    elif RTCL < SCL and RTCL <= nonRTCL:
        MC=RTCL
        
        iat_RT=return_time(iat_RT_mean) 
        RTCL=MC+iat_RT        
        
        if s==1:
            nRT+=1
            
        elif s==0:
            st_RT=return_time(st_RT_mean)
            SCL=MC+st_RT
            s=1
            
        elif s==2:
            nNonRT+=1
            rem_time=SCL-MC
            s=1
            
            st_RT=return_time(st_RT_mean)
            SCL=MC+st_RT
     
    elif nonRTCL <= SCL or nonRTCL < RTCL:
        MC=nonRTCL
        
        iat_NRT=return_time(iat_NRT_mean) 
        nonRTCL=MC+iat_NRT
        
        if s==0:
            st_NRT=return_time(st_NRT_mean)
            SCL=MC+st_NRT
            s=2  
            
        if s==1 or s==2:
            nNonRT+=1    
            
    row=[round(MC,4),round(RTCL,4),round(nonRTCL,4),nRT,nNonRT,round(SCL,4),s,round(rem_time,4)]
    data.append(row)

df = pd.DataFrame(data, columns=["MCL","RTCL","non-RTCL","N(RT)","N(nonRT)","SCL","s","remaining time"])

df.loc[df['SCL'] == max_iter+1000, 'SCL'] = np.nan
print(df.to_string())

          MCL      RTCL  non-RTCL  N(RT)  N(nonRT)       SCL  s  remaining time
0      0.0000    3.0000    5.0000      0         0    4.0000  2          0.0000
1      3.0000   14.2753    5.0000      0         1    6.7829  1          1.0000
2      5.0000   14.2753    7.1467      0         2    6.7829  1          1.0000
3      6.7829   14.2753    7.1467      0         1    7.7829  2          0.0000
4      7.1467   14.2753   20.2719      0         2    7.7829  2          0.0000
5      7.7829   14.2753   20.2719      0         1   10.2783  2          0.0000
6     10.2783   14.2753   20.2719      0         0   14.3022  2          0.0000
7     14.2753   42.7486   20.2719      0         1   15.6321  1          0.0269
8     15.6321   42.7486   20.2719      0         0   15.6590  2          0.0000
9     15.6590   42.7486   20.2719      0         0       NaN  0          0.0000
10    20.2719   42.7486   36.6896      0         1   23.6140  2          0.0000
11    23.6140   42.7486   36.6896      0

In [6]:
df.to_csv("Output_task_2.2.csv", sep='\t')